In [12]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [13]:
#reading in the csv file
staten_island_df = pd.read_csv('NYCCrimeData_Parks_Staten_Island_TableToExcel.csv')
staten_island_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 62 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   OBJECTID              390 non-null    int64  
 1   CMPLNT_NUM            366 non-null    object 
 2   CMPLNT_FR_DT          366 non-null    object 
 3   CMPLNT_FR_TM          366 non-null    object 
 4   CMPLNT_TO_DT          336 non-null    object 
 5   CMPLNT_TO_TM          366 non-null    object 
 6   ADDR_PCT_CD           364 non-null    float64
 7   RPT_DT                366 non-null    object 
 8   KY_CD                 366 non-null    float64
 9   OFNS_DESC             366 non-null    object 
 10  PD_CD                 348 non-null    float64
 11  PD_DESC               366 non-null    object 
 12  CRM_ATPT_CPTD_CD      366 non-null    object 
 13  LAW_CAT_CD            366 non-null    object 
 14  BORO_NM               366 non-null    object 
 15  LOC_OF_OCCUR_DESC     3

In [14]:
#dividing the df into crimes and public institutions
#the letter r here is used as the official abbreviation for staten island

r_crimes = staten_island_df.iloc[0:365, :]
r_crimes = r_crimes.drop(columns= ["Borough", "Public Area"])

KeyError: "['Borough', 'Public Area'] not found in axis"

In [15]:
r_crimes.head()

,OBJECTID,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,...,nta2010,nta2020,council,ct2010,ct2020,borocode,schooldist,policeprct,datasource,uid
0,1,261214832,2023-01-01,22:55:00,2023-01-01,23:00:00,120.0,2023-01-01,105.0,ROBBERY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,261233859,2023-01-02,13:20:00,2023-01-02,13:33:00,121.0,2023-01-02,105.0,ROBBERY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,261233841,2023-01-02,8:00:00,2023-01-02,09:15:00,120.0,2023-01-02,105.0,ROBBERY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,261261343,2023-01-03,1:14:00,2023-01-03,01:16:00,121.0,2023-01-03,105.0,ROBBERY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,261347872,2023-01-04,14:00:00,2023-01-04,14:02:00,120.0,2023-01-04,105.0,ROBBERY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#each location a crime took place

r_crimes_gdf = gpd.GeoDataFrame(
    r_crimes,
    geometry=gpd.points_from_xy(
        r_crimes["Longitude"],
        r_crimes["Latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

# each public institution

r_public_gdf = gpd.GeoDataFrame(
    r_public, 
    geometry=gpd.points_from_xy(
        r_public["Longitude"],
        r_public["Latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

# EPSG:3857 converts it to meters

r_crimes_gdf = r_crimes_gdf.to_crs({"init": "EPSG:3857"})
r_public_gdf = r_public_gdf.to_crs({"init": "EPSG:3857"})

# 1 miles would be 1609 meters

x = r_public_gdf.buffer(1609).unary_union

crimes_per = r_crimes_gdf["geometry"].intersection(x)

# print all the nearby points
all_crimes_near_pi = (r_crimes_gdf[~crimes_per.is_empty])
all_crimes_near_pi

NameError: name 'gpd' is not defined

In [17]:
#create empty list to hold the number of crimes commited within a 1 mile radius of each public institution
crimes_per_location = []

# Define the radius in meters
radius = 1609 # about 1 mile

for offense in r_crimes_gdf["Offense"]:
    crimes_per_location = []
    r_crimes_gdf_each = r_crimes_gdf[r_crimes_gdf["Offense"] == offense]

    for i in range(len(r_public_gdf)):

        # Calculate the distances between all pairs of points
        distances = gpd.GeoSeries(r_crimes_gdf_each.geometry).distance(r_public_gdf.geometry.iloc[i])

        # Filter the pairs where the distance is within the radius
        within_radius = distances <= radius

        # Count the number of points within the radius for each point in gdf2
        count_within_radius = within_radius.sum()

        crimes_per_location.append(count_within_radius)
    r_public_gdf[offense] = crimes_per_location

NameError: name 'r_crimes_gdf' is not defined

In [18]:
r_public_gdf["Total Crimes"] = r_public_gdf[['ROBBERY', 'MURDER & NON-NEGL. MANSLAUGHTER', 'RAPE']].sum(axis=1)
r_public_gdf

NameError: name 'r_public_gdf' is not defined